In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/10.kit_info'):
    shutil.rmtree('./file/10.kit_info')

os.makedirs('./file/10.kit_info')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/8.kit_part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/8.kit_part/{file}',
                           header=0,
                           dtype=str).fillna('')

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1

                    try:
                        request_url = input_.loc[a, 'Kit_Info']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                        
                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))
                
                            if html.xpath('//span[@class="la-btn-v"]/text()')[0] == 'Continue Shopping':
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                dict_specification, list_tr, dict_count = {}, html.xpath('//table[@class="moreinfotable"]/tr')[1:] if html.xpath('//table[@class="moreinfotable"]/tr') else [], {}
                for i, tr in enumerate(list_tr):
                    list_td = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
                    list_td = [re.sub(r' +', ' ', td).strip() for td in list_td]
                    if list_td[0] in dict_count:
                        dict_count[list_td[0]] += 1
                    else:
                        dict_count[list_td[0]] = 1
                
                dict_no = {}
                for i, tr in enumerate(list_tr):
                    dict_specification[str(i)] = {}
                
                    list_td = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
                    list_td = [re.sub(r' +', ' ', td).strip() for td in list_td]
                    for j, td in enumerate(list_td):
                        if j == 0:
                            if dict_count[td] == 1:
                                dict_specification[str(i)][str(j)] = td
                            else:
                                if td in dict_no:
                                    dict_no[td] += 1
                                else:
                                    dict_no[td] = 1
                                dict_specification[str(i)][str(j)] = f'''{td}-{dict_no[td]}'''
                                    
                        else:
                            dict_specification[str(i)][str(j)] = td

                # = = = = = = = = = = = = = = =

                dict_kit, list_tr = {}, html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or (contains(@class, "altrow-b-1")))]/tr')
                for i, tr in enumerate(list_tr):
                    dict_kit[str(i)] = {}
                
                    list_td = tr.xpath('./td')
                    dict_kit[str(i)]['Kit_Quantity'] = int(list_td[0].xpath('./text()')[0].strip())
                
                    # = = = = = = = = = = = = = = =
                
                    dict_kit[str(i)]['Kit_Type'] = list_td[1].xpath('./text()')[0].strip()
                
                    # = = = = = = = = = = = = = = =
                
                    dict_ = json.loads(list_td[2].xpath('./@value')[0])
                    dict_kit[str(i)]['Kit_Type Code'] = dict_['parttype']
                    dict_kit[str(i)]['Kit_Part Code'] = dict_['partkey']
                
                    # = = = = = = = = = = = = = = =
                
                    dict_2 = json.loads(tr.xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
                    dict_kit[str(i)]['Kit_Part Number'] = dict_2['partnumber']
                    dict_kit[str(i)]['Kit_Manufacturer'] = dict_2['catalogname']
                
                    # = = = = = = = = = = = = = = =
                
                    dict_kit[str(i)]['Kit_Url'] = f'''https://www.rockauto.com/en/parts/{dict_kit[str(i)]['Kit_Manufacturer']},{dict_kit[str(i)]['Kit_Part Number']},{dict_kit[str(i)]['Kit_Type']},{dict_kit[str(i)]['Kit_Type Code']}'''
                
                    # = = = = = = = = = = = = = = =
                
                    list_info = tr.xpath('./div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
                    dict_kit[str(i)]['Kit_Info'] = list_info[0].xpath('./@href')[0].strip() if list_info else ''
                    dict_kit[str(i)]['Kit_Info'] = dict_kit[str(i)]['Kit_Info'] if dict_kit[str(i)]['Kit_Info'].startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={dict_kit[str(i)]['Kit_Part Code']}&cc=0&pt={dict_kit[str(i)]['Kit_Type Code']}&Lennon=1'''
                
                    # = = = = = = = = = = = = = = =
                    
                    list_src = list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]')
                    dict_ = json.loads(list_src[0].xpath('./@value')[0]) if list_src else {}
                
                    dict_src = {}
                    if dict_:
                        list_src = dict_['Slots']
                        for j, src in enumerate(list_src):
                            dict_src[str(j)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
                
                    dict_kit[str(i)]['Kit_Json_Src'] = json.dumps(dict_src)
                
                    # = = = = = = = = = = = = = = =
                
                    dict_kit[str(i)]['Kit_JOIN_MPNTCPC'] = f'''{dict_kit[str(i)]['Kit_Manufacturer']};{dict_kit[str(i)]['Kit_Part Number']};{dict_kit[str(i)]['Kit_Type Code']};{dict_kit[str(i)]['Kit_Part Code']}'''

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'Kit_JOIN_MPNTCPC': input_.loc[a, 'Kit_JOIN_MPNTCPC'],
                                         'Kit_Type': input_.loc[a, 'Kit_Type'],
                                         'Kit_Manufacturer': input_.loc[a, 'Kit_Manufacturer'],
                                         'Kit_Part Number': input_.loc[a, 'Kit_Part Number'],
                                         'Kit_Vehicle': input_.loc[a, 'Kit_Vehicle'],
                                         'Kit_OEM': input_.loc[a, 'Kit_OEM'],
                                         'Kit_Picture': input_.loc[a, 'Kit_Picture'],
                                         'Kit_Url': input_.loc[a, 'Kit_Url'],
                                         'Kit_Json_Src': input_.loc[a, 'Kit_Json_Src'],
                                         'Kit_Info': input_.loc[a, 'Kit_Info'],
                                         'Kit_Choose': input_.loc[a, 'Kit_Choose'],
                                         'Kit_Json_Price': input_.loc[a, 'Kit_Json_Price'],
                                         'Kit_Json_Note 1': input_.loc[a, 'Kit_Json_Note 1'],
                                         'Kit_Json_Note 2': input_.loc[a, 'Kit_Json_Note 2'],
                                         'Kit_Json_Specification': json.dumps(dict_specification),
                                         'Kit_Json_Kit': json.dumps(dict_kit),
                                         'Kit_Type Code': input_.loc[a, 'Kit_Type Code'],
                                         'Kit_Part Code': input_.loc[a, 'Kit_Part Code']}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Kit_JOIN_MPNTCPC']} > {request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Kit_JOIN_MPNTCPC'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/10.kit_info/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Kit_JOIN_MPNTCPC'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-kit_info_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/10.kit_info'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/10.kit_info/{file}',
              f'''./file/10.kit_info/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：41

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > MELLING;470;5724;10110008 > https://www.rockauto.com/en/moreinfo.php?pk=10110008&cc=0&pt=5724
[剩余数量：36] - [当前时间：19:10:41]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > MELLING;1025;5724;8921164 > https://www.rockauto.com/en/moreinfo.php?pk=8921164&cc=0&pt=5724
[剩余数量：35] - [当前时间：19:10:41]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > MELLING;715F;5724;8306448 > https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724
[剩余数量：34] - [当前时间：19:10:42]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > MELLING;722F;5724;9659040 > https://www.rockauto.com/en/moreinfo.php?pk=9659040&cc=0&pt=5724
[剩余数量：33] - [当前时间：19:10:43]

[状态：ok，尝试次数：2] - 100.00% - crawler_1 > MELLING;717F;5568;8527968 > https://www.rockauto.com/en/moreinfo.php?pk=8527968&cc=0&pt=5568
[剩余数量：32] - [当前时间：19:10:44]

[状态：ok，尝试次数：2] - 100.00% - crawler_1 > MELLING;BG5053;5728;9659056 > https://www.rockauto.com/en/moreinfo.php?pk=9659056&cc=0&pt=5728
[剩余数量：31] - [当前时间：19:10:44]

[状态：ok，尝试次数：1] - 100.00

Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 2794.34it/s]

Done ~
